<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mHdr

A large percentage of the Montage modules focus on processing a set of images (often retrieved from a mission archive); projecting them to a common frame, adjusting their background levels as a set, and coadding them into a mosaic.

This page is focused on the details of one of these modules: mHdr.  For a broader context, please see <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands.

<b>Note:</b> The MontagePy python package has no external dependencies. We include packages like AstroPy on this page to aid in visualizing MontagePy package results.


In [1]:
from MontagePy.main import mHdr, mViewer

help(mHdr)

Help on built-in function mHdr in module MontagePy.main:

mHdr(...)
    mHdr creates a header template file from location, size, resolution and rotation inputs. In order to support object name lookup for the location string argument, this service must have a network connection available.
    
    Parameters
    ----------
    locstr : str
        A (quoted if necessary) string containing a coordinate or the name of an object on the sky.
    width : float
        Image width in degrees.
    height : float
        Image height in degrees.
    outfile : str
        Output FITS header file.
    csys : str, optional
        Coordinate system string (e.g. 'EquJ', 'Galactic', etc. Fairly forgiving.
    equinox : float, optional
        Coordinate system equinox (e.g. 2000.0).
    resolution : float, optional
        Image pixel resolution (in arcsec).
    rotation : float, optional
        Image rotation on the sky.
    band2MASS : str, optional
        Optional argument when mosaicking 2MASS

## mHdr Example:

Much of the Montage processing keys off a FITS header template.  Sometimes these headers are extracted from a reference file (mGetHdr).  Sometimes they are generated as a bounding box around a catalog of sources or other images (mMakeHdr) and sometimes there are just created manually with an editor.

mHdr can be used to build a simple (TAN projection) header given a location (coordinates or object name) and size.

To use mHdr with object names you must have a working network connection.  Name resolution is done via a remote service.  Width and height are in degrees and there are overrides for pixel scale (1 arcsecond by default), coordinate system (Equatorial J2000 by default) and rotation.

In [2]:
import os

try:
    os.mkdirs('work/M17')
except:
    pass

rtn = mHdr("M17", 1.0, 1.0, 'work/M17/M17.hdr')

print(rtn)

{'status': '0', 'count': 16}


The output header looks like this:

In [3]:
with open("work/M17/M17.hdr", "r") as file:
  for line in file:
    print(line, end='')

SIMPLE  = T
BITPIX  = -64
NAXIS   = 2
NAXIS1  = 3601
NAXIS2  = 3600
CTYPE1  = 'RA---TAN'
CTYPE2  = 'DEC--TAN'
CRVAL1  =  275.196290
CRVAL2  =  -16.171530
CRPIX1  = 1801.000000
CRPIX2  = 1800.500000
CDELT1  =   -0.000277778
CDELT2  =    0.000277778
CROTA2  =    0.000000
EQUINOX = 2000.0
END


## mHdr Error Handling

If mHdr encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user asks for an unsupported dataset:

In [4]:
rtn = mHdr("dummy object", 1., 1., "output/bad.hdr")

print(rtn)

{'status': '1', 'msg': b'Invalid object/coordinates'}


&nbsp;

## Classic Montage: mHdr as a Stand-Alone Program

### mHdr Unix/Windows Command-line Arguments

<p>mHdr can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mHdr [-c system] [-e equinox] [-h height(deg)] [-p pixsize(arcsec)] [-r rotation] [-t 2mass-band] object|location width(deg) outfile (object/location must be a single argument string)
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mHdr can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/* mHdr -- Create a header template file from location, size, resolution */
/* and rotation inputs. In order to support object name lookup for the   */
/* location string argument, this service must have a network connection */
/* available.                                                            */
/*                                                                       */
/*   char  *locstr         A (quoted if necessary) string containing     */
/*                         a coordinate or the name of an object on      */
/*                         the sky                                       */
/*                                                                       */
/*   double width          Image width in degrees                        */
/*   double height         Image height in degrees                       */
/*                                                                       */
/*   char  *outfile        Output FITS header file                       */
/*                                                                       */
/*   char  *csys           Coordinate system string (e.g. 'EquJ',        */
/*                         'Galactic', etc. Fairly forgiving.            */
/*                                                                       */
/*   double equinox        Coordinate system equinox (e.g. 2000.0)       */
/*   double resolution     Image pixel resolution (in arcsec)            */
/*   double rotation       Image rotation on the sky                     */
/*                                                                       */
/*   char  *band2MASS      Optional argument when mosaicking 2MASS, adds */
/*                         proper MAGZP value                            */
/*                                                                       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mHdrReturn *mHdr(char *locstr, double width, double height, char *outfile, char *csys, double equinox, 
                        double resolution, double rotation, char *band2MASS, int debug)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mHdrReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   int    count;         // Number of lines in output file.
};
</pre>